# Bestmögliches Regressionsmodell - kaggle-Contest

Challenge: cml1/3Db Immobilienrechner <br/>
Team: Alexander Shanmugam, Si Ben Tran, Gabriel Torrez Gamez, Haris Alic <br/>
Aufgabe: 2.2 Bestmögliches Regressionsmodell - kaggle-Contest

Entwickle mit beliebigen Algorithmen das bestmögliche Modell im Sinne des Mean absolute percentage error (MAPE). Vegleiche dabei mindestens drei algorithmische Ansätze, wobei ein multiples lineares Modell Teil davon sein soll als Benchmark. Untersuche die ‘Variable Importance’ für dein bestes Modell.

Abgabe

Notebook und daraus erstellter Bericht (ohne Code) als pdf, welche die Entwicklung deines besten Modells, sowie der zwei weiteren Modelle dokumentiert, inklusive verwendeter Features, Preprocessing, Model Selection Prozess und Untersuchung der ‘Variable Importance’.

Eingabe der Vorhersage des Preises für den Testdatensatz mit deinem bestmöglichen Modell auf kaggle.



# Module importieren

Hier in diesem Abschnitt importieren wir die wichtigisten Module, die wir für die weitere Bearbeitung unserer Machine Learning Modelle benötigen, um die Vorhersage des Immobilienpreises zu erstellen.

In [1]:
# Basic Data Science modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn modules
## linear_model
from sklearn.linear_model import LinearRegression

## model_selection
from sklearn.model_selection import train_test_split, GridSearchCV

## metrics
from sklearn.metrics import mean_absolute_percentage_error


# Daten importieren

Hier in diesem Abschnitt importieren wir die Immobilien Daten, die uns Fernando zur Verfügung gestellt hat.

v1 ist der von uns bearbeitete und bereinigter Datensatz 

v2 ist der von Fernando bearbeitete und bereinigter Datensatz 

In [2]:
# url and relative path for the data

rel_path_v1 = "https://raw.githubusercontent.com/Immobilienrechner-Challenge/data/main/clean_gde.csv"
# Clean data einlesen
rel_path_v2 = "../data/immo_data_202208_v2.csv"

# Read data into pandas dataframe
raw_df_v1 = pd.read_csv(rel_path_v1, index_col=0)
raw_df_v2 = pd.read_csv(rel_path_v2)

C:\Users\Si Ben Tran\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Si Ben Tran\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Daten vorbereiten

Hier werden die Daten vorbereitet, indem wir die Daten in Trainings- und Validierungsdaten aufteilen.
Die Testdaten befinden sich bereits auf der Kaggle Plattform und werden von Fernando zur Verfügung gestellt.

In [3]:
# drop the columns municipality, street, street_nr and zip_code
df_v1 = raw_df_v1.drop(columns=["municipality", "street", "street_nr", "zip_code", "availability"])

# count number of dtypes
print("Number of dtypes:", df_v1.dtypes.value_counts())

# print dtypes object columns
print("Dtypes object columns:", df_v1.select_dtypes(include="object").columns)

# make one hot encoding for the columns objects
df_v1 = pd.get_dummies(df_v1, columns = ['canton', 'type'])

# print shape of df_v1
print("Shape of df_v1:", df_v1.shape) 

# get top 10 cols with highest NA values
print("Top 10 cols with highest NA values:", df_v1.isna().sum().sort_values(ascending=False).head(30))


Number of dtypes: float64    54
object      2
int64       1
dtype: int64
Dtypes object columns: Index(['canton', 'type'], dtype='object')
Shape of df_v1: (21424, 124)
Top 10 cols with highest NA values: floor_space                18551
plot_area                  16586
floor                      15938
gde_politics_pda           10206
rooms                       8910
gde_politics_bdp            8207
gde_politics_rights         6156
gde_politics_glp            5340
gde_politics_evp            4153
living_space                1504
price                       1031
gde_politics_gps             495
gde_politics_cvp             334
gde_politics_fdp             157
gde_politics_sp              102
gde_politics_svp              55
canton_UR                      0
type_Farm house                0
canton_SO                      0
canton_SZ                      0
canton_TG                      0
type_Hobby room                0
type_Granny flat               0
type_Furnished dwelling        0
type_

## Daten V1 

Hier bearbeiten wir die Daten, die wir anfang 3. Semester erhalten haben.

In [4]:
# splitting dataframe into train and test set
train_v1, test_v1 = train_test_split(df_v1, test_size=0.2, random_state=42)

# splitting train_v1 and test_v1 into X and y 
X_train_v1 = train_v1.drop("price", axis=1)
y_train_v1 = train_v1["price"]

X_test_v1 = test_v1.drop("price", axis=1)
y_test_v1 = test_v1["price"]

# print shape of X_train_v1, y_train_v1, X_test_v1, y_test_v1
print("Shape of X_train_v1: ", X_train_v1.shape)
print("Shape of y_train_v1: ", y_train_v1.shape)
print("Shape of X_test_v1: ", X_test_v1.shape)
print("Shape of y_test_v1: ", y_test_v1.shape)

Shape of X_train_v1:  (17139, 123)
Shape of y_train_v1:  (17139,)
Shape of X_test_v1:  (4285, 123)
Shape of y_test_v1:  (4285,)


## Daten V2

Hier bearbeiten wir die Daten, die wir von Fernando in der Mitte des Semester (18.11.2022) erhalten haben. <br/>
Die Dokumentation der Features ist unter folgenden Link zu finden: [Link](www.gooogle.com)

In [5]:
# CODE

# Modell 1 - Multiple Lineare Model 

Modell 1 ist ein multiples lineares Modell, welches wir als Benchmark für die weiteren Modelle verwenden werden. Wir haben uns für dieses Modell entschieden, da es ein einfaches Modell ist, welches wir schnell erstellen können und es uns ermöglicht, die Vorhersage des Preises zu erstellen.

In [35]:
# create a multiple linear Regressionmodell with gridsearch
# create a dictionary with the parameters
params = {"fit_intercept": [True, False],
            "normalize": [True, False]}

# create a gridsearch object
grid_search = GridSearchCV(LinearRegression(), params, cv=5, verbose=2, n_jobs=-1, scoring="neg_mean_absolute_percentage_error")
# fit the gridsearch object
grid_search.fit(X_train_v1, y_train_v1)

# print the best parameters
print("Best parameters:", grid_search.best_params_)
# print the best score
print("Best score:", grid_search.best_score_)
# print the best estimator
print("Best estimator:", grid_search.best_estimator_)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\Si Ben Tran\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Si Ben Tran\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Si Ben Tran\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py", line 662, in fit
    X, y = self._validate_data(
  File "C:\Users\Si Ben Tran\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py", line 581, in 

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# Modell 2 - Lasso Regression

Modell 2 ist ein multiples lineares Lasso Modell. 

In [ ]:
# CODE

# Modell 3 - Ridge Regression

Modell 3 ist ein multiples lineares Ridge Modell.

In [ ]:
# CODE

# Modell 4 - Elastic Net Regression

Modell 4 ist ein multiples lineares Elastic Net Modell.

In [ ]:
# CODE

# Modell 5 - KNN Regression

Modell 5 ist ein KNN Modell.

In [ ]:
# Code

# Modell 6 - Decision Tree Regression

Modell 6 ist ein Decision Tree Modell.

In [ ]:
# CODE

# Modell 7 - Random Forest Regression

Modell 7 ist ein Random Forest Modell.

In [5]:
# CODE

# Modell 8 - Gradient Boosting Regression

Modell 8 ist ein Gradient Boosting Modell.

In [6]:
# CODE

# Modell 9 - XGBoost Regression

Modell 9 ist ein XGBoost Modell.

In [ ]:
# CODE

# Vergleiche der Modelle

Hier werden die Modelle miteinander verglichen und tabellerisch sowie visualisiert dargestellt.
Dabei dient das Modell 1 - Multiple Linear Model, als Benchmark.

In [ ]:
# CODE

# Untersuchung der "Variable Importance" für das beste Modell

Hier untersuchen wir die "Variable Importance" für das beste Modell.
Es hat sich herausgestellt, dass die .....

In [ ]:
# CODE

# Fazit

Hier schreiben wir unser Fazit.